In [ ]:
import os
import sys
import time
import re
import copy
import pandas as pd
from tqdm import tqdm
from selenium import webdriver

# set utils
util_path = os.pardir
if util_path not in sys.path:
    sys.path.append(os.pardir)
from utils.helper import load_config
from crawler import Crawler, Utills

# load config file
Project_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
CONFIG_PATH = "config.yaml"
config = load_config(os.path.join(Project_PATH, 'jobkorea', CONFIG_PATH))
raw_data = pd.read_excel('job_korea_step1.xlsx', engine='openpyxl')
init_url = config['init_url']
selen_path_dict = config['selen_path']
solutions = config['solution']

# chrome option
chrome_opt = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_opt.add_experimental_option("prefs", prefs) 
chrome_opt.add_experimental_option("detach", True) # prevent automatically closed tab
chrome_opt.add_experimental_option("excludeSwitches", ["enable-logging"])

#location
location = "home"


if location == 'home':
    crawler = Crawler(selen_path = selen_path_dict['home'],
                        chromeOption=chrome_opt,
                        init_url=init_url,
                        solutions = solutions)
elif location == 'laptop':
    crawler = Crawler(selen_path = selen_path_dict['laptop'],
                        chromeOption=chrome_opt,
                        init_url=init_url,
                        solutions = solutions)
elif location == 'khrrc':
    crawler = Crawler(selen_path = selen_path_dict['khrrc'],
                        chromeOption=chrome_opt,
                        init_url=init_url, 
                        solutions = solutions)
else:
    raise NotImplementedError

In [ ]:
crawler_utils = Utills(raw_data=raw_data)
raw_data.shape

In [ ]:
for idx, corp_url in tqdm(enumerate(raw_data['href'])):
    if idx < 10:
        rs = crawler.search_corporation(corp_url)
        raw_data.loc[idx, '매출수'] = rs['매출수']
        raw_data.loc[idx, '사원수'] = rs['사원수']
        raw_data.loc[idx, '설립일'] = rs['설립일']
        
        if idx % 500 == 99:
            time.sleep(60) 

    else:
        pass

raw_data.to_excel(os.path.join(Project_PATH, 'jobkorea', 'result', 'rs_corp_info_1000.xlsx'))
